Creating MEDS files
https://github.com/esheldon/meds/wiki/Creating-MEDS-Files-in-Python

In [1]:
import numpy as np
import pandas as pd

import sklearn.neighbors as neighbors

from importlib import reload
import copy
import synthetic.render.generator as gen

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import multiprocessing as mp

import sys
import os
import math
import logging
import time
import galsim
import fitsio as fio

import ngmix

import synthetic.render.render as render
import synthetic.render.icl as icl
import synthetic.render.frame as frame

import images
import meds
import subprocess

In [2]:
catalogs = []
catalog_tag = '/e/ocean1/users/vargatn/DES/SIM_DATA/SKY_MODEL_EPSILON/res_clust_v2_z0l0/epsilon_clust_v8-z0l0_15000_{:03d}.fits'
for i in np.arange(1):
    fname = catalog_tag.format(i)
    data = fio.read(fname)
    catalog_cluster = data[data['LOC'] == 1]
    catalog_field = data[data['LOC'] == 0]
    catalog = np.hstack((catalog_cluster, catalog_field))
    catalog['G1'] = 0.3
    catalog['G2'] = -0.2
    catalogs.append(catalog)

In [5]:

i = 0
name = 'canvas_{:03d}'.format(i)
print(name)
fr = frame.Frame(catalog, name=name, noise_std=4)
fr.render()
fr.extract()
fr.ksb()


canvas_000
starting xshear calculations in 100 processes


In [15]:
fr.scat.shape

(9323,)

In [8]:
meds.util.get_meds_input_struct(10)

array([(-9999, -9999, -9999., -9999.), (-9999, -9999, -9999., -9999.),
       (-9999, -9999, -9999., -9999.), (-9999, -9999, -9999., -9999.),
       (-9999, -9999, -9999., -9999.), (-9999, -9999, -9999., -9999.),
       (-9999, -9999, -9999., -9999.), (-9999, -9999, -9999., -9999.),
       (-9999, -9999, -9999., -9999.), (-9999, -9999, -9999., -9999.)],
      dtype=[('id', '<i8'), ('box_size', '<i8'), ('ra', '<f8'), ('dec', '<f8')])

In [7]:
meds.util.get_meds_output_struct(10, 1, )

array([(-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-

In [18]:
"""
column             type     description
----------------------------------------------------------------------
id                 i8       The id column from the input file
ncutout            i8       number of cutouts for this object
box_size           i8       box size for each cutout
file_id            i8[NMAX] zero-offset id into the file names in the 
                            image_info extension
start_row          i8[NMAX] zero-offset, start of cutout in the 
                            image extensions.
orig_row           f8[NMAX] zero-offset position in original image      FLOAT the position within the image EXACT
orig_col           f8[NMAX] zero-offset position in original image      FLOAT the position within the image EXACT
orig_start_row     i8[NMAX] zero-offset start corner in original image  INT This is the starting corner in row
orig_start_col     i8[NMAX] zero-offset start corner in original image  INT This is the starting corenr is col
cutout_row         f8[NMAX] zero-offset position in cutout imag         FLOAT the position within the cutout EXACT
cutout_col         f8[NMAX] zero-offset position in cutout image        FLOAT the position within the cutout EXACT
dudrow             f8[NMAX] jacobian of transformation                      
dudcol             f8[NMAX] row,col->ra,dec tangent plane (u,v)
dvdrow             f8[NMAX] Note Source Extractor uses y for row
dvdcol             f8[NMAX]
"""

'\ncolumn             type     description\n----------------------------------------------------------------------\nid                 i8       The id column from the input file\nncutout            i8       number of cutouts for this object\nbox_size           i8       box size for each cutout\nfile_id            i8[NMAX] zero-offset id into the file names in the \n                            image_info extension\nstart_row          i8[NMAX] zero-offset, start of cutout in the \n                            image extensions.\norig_row           f8[NMAX] zero-offset position in original image      FLOAT the position within the image EXACT\norig_col           f8[NMAX] zero-offset position in original image      FLOAT the position within the image EXACT\norig_start_row     i8[NMAX] zero-offset start corner in original image  INT This is the starting corner in row\norig_start_col     i8[NMAX] zero-offset start corner in original image  INT This is the starting corenr is col\ncutout_row     

In [25]:
# ! scp /home/moon/vargatn/DES/Y3_WORK/Y3_SIM/sim-v1-000-r.fits .
# ! scp /home/moon/vargatn/DES/Y3_WORK/Y3_SIM/truth_catalog-v1-000-r.dat .

In [26]:
m=meds.MEDS("./sim-v1-000-r.fits")

In [30]:
m.get_jacobian(0, 0)

{'row0': 24.01335112238303,
 'col0': 24.23530559753999,
 'dudrow': 0.145818665898048,
 'dudcol': -0.2152601139916856,
 'dvdrow': -0.2152601139916856,
 'dvdcol': -0.145818665898048}

In [31]:
j = m.get_jacobian_matrix(0, 0)

In [35]:
cat = m.get_cat()

In [41]:

for name in np.array(cat.dtype.names):
    print(name)
    print(cat[name])

id
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57]
number
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57]
ra
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
dec
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
box_size
[48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48
 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48 48
 48 48 48 48 48 48 48 48 48 48]
ncutout
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 

In [46]:
cat["start_"]

array([[     0,   9999,   9999,   9999,   9999,   9999,   9999,   9999,
          9999,   9999,   9999],
       [  2304,   9999,   9999,   9999,   9999,   9999,   9999,   9999,
          9999,   9999,   9999],
       [  4608,   9999,   9999,   9999,   9999,   9999,   9999,   9999,
          9999,   9999,   9999],
       [  6912,   9999,   9999,   9999,   9999,   9999,   9999,   9999,
          9999,   9999,   9999],
       [  9216,   9999,   9999,   9999,   9999,   9999,   9999,   9999,
          9999,   9999,   9999],
       [ 11520,   9999,   9999,   9999,   9999,   9999,   9999,   9999,
          9999,   9999,   9999],
       [ 13824,   9999,   9999,   9999,   9999,   9999,   9999,   9999,
          9999,   9999,   9999],
       [ 16128,   9999,   9999,   9999,   9999,   9999,   9999,   9999,
          9999,   9999,   9999],
       [ 18432,   9999,   9999,   9999,   9999,   9999,   9999,   9999,
          9999,   9999,   9999],
       [ 20736,   9999,   9999,   9999,   9999,   9999,

In [48]:

m = meds.MEDS('gauss_uberseg_meds.fits.fz')

In [52]:
cat = m.get_cat()

In [54]:
for name in np.array(cat.dtype.names):
    print(name)
    print(cat[name][:2])

id
[1 2]
box_size
[32 32]
ra
[199.2721425  199.50242942]
dec
[32.97420924 32.97501918]
ncutout
[1 9]
file_id
[[    0 -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999
  -9999 -9999 -9999 -9999 -9999 -9999 -9999]
 [    0     4     6     7    10    11    12    13    17 -9999 -9999 -9999
  -9999 -9999 -9999 -9999 -9999 -9999 -9999]]
start_row
[[    0 -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999
  -9999 -9999 -9999 -9999 -9999 -9999 -9999]
 [ 1024  2048  3072  4096  5120  6144  7168  8192  9216 -9999 -9999 -9999
  -9999 -9999 -9999 -9999 -9999 -9999 -9999]]
orig_row
[[   51.1285903  -9999.         -9999.         -9999.
  -9999.         -9999.         -9999.         -9999.
  -9999.         -9999.         -9999.         -9999.
  -9999.         -9999.         -9999.         -9999.
  -9999.         -9999.         -9999.        ]
 [   66.55291212    11.4958072     63.4958072     17.4958072
     49.4958072     37.4958072     56.4958072     61.4958072
     51.

In [55]:
32*32

1024